In [204]:
# Import necessary libraries
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
from scipy import signal

In [205]:
class Pixel():
    '''
    Class for keeping pixel cooridinate
    '''

    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.point = (self.x, self.y)

    def __repr__(self):
        return f'({self.x}, {self.y})'

    def adjust_width(self, width):
        x = self.x + width
        return (x, self.y)


In [206]:
class Harris():
    '''
    Class for implimenting Harris Corner Detection.
    '''

    K_THRESHOLD = 0.1
    
    def __init__(self):
        pass

    def haar_filter(self, sigma):
        '''
        Create Haar filters.
        Input (simga):
            sigma: value of sigma
        Output [haar_x, haar_y]:
            haar_x: Haar filter in x direction
            haar_y: Haar filter in y direction
        '''
        sigmax4 = int(sigma * 4)
        if sigmax4 % 2 == 1:
            sigmax4 = sigmax4 + 1
        haar_x = np.ones((sigmax4, sigmax4))
        haar_x[:, :int(sigmax4/2)] = -1
        haar_y = haar_x.T * -1

        return [haar_x, haar_y]

    def find_harris_corners(self, imgBRG, sigma):
        '''
        Apply Harris Corner Detector on gray scale
        Input (image, simga):
            imgBRG: original image in BGR format load by CV2
            sigma: value of sigma
        Output [corners]:
            corners: list of corner's coordinates
        '''
        image = cv.cvtColor(imgBRG, cv.COLOR_BGR2GRAY)
        h, w = image.shape
        [haar_x, haar_y] = self.haar_filter(sigma)
        dx = signal.convolve2d(image, haar_x, mode='same')
        dy = signal.convolve2d(image, haar_y, mode='same')
        dx2 = dx * dx
        dy2 = dy * dy
        dxy = dx * dy

        kernel_width = int(np.ceil(sigma * 5))
        if kernel_width % 2 == 0:
            kernel_width = kernel_width + 1
        kernel = np.ones((kernel_width, kernel_width))
        
        sum_dx2 = signal.convolve2d(dx2, kernel, mode='same')
        sum_dy2 = signal.convolve2d(dy2, kernel, mode='same')
        sum_dxy = signal.convolve2d(dxy, kernel, mode='same')

        trace = sum_dx2 + sum_dy2
        trace2 = trace * trace
        det = (sum_dx2 * sum_dy2) - (sum_dxy * sum_dxy)
        r = det - self.K_THRESHOLD * trace2

        # Search for max r value pixel in 41 x 41 pixel window. Keep only positive value
        corners = []
        width = 20 # 41 / 2
        for i in range(width, w - width):
            for j in range(width, h - width):
                window = r[j - width : j + width+1, i - width : i + width+1]
                max_value = np.amax(window)
                if max_value > 0 and r[j, i] == max_value:
                    corners.append(Pixel(i, j))
        
        return corners

In [207]:
class Image(Harris):
    ''' 
    Class for store images and related parameters.
    '''

    FILEPATH = 'ece661_pics\\hw4_image\\'
    FILETYPE = '.png'

    def __init__(self, file1, file2, savename):
        self.file1 = file1
        self.file2 = file2
        self.load_images()
        self.savename = savename

    def load_images(self):
        self.image1 = cv.imread(self.FILEPATH + self.file1)
        self.image2 = cv.imread(self.FILEPATH + self.file2)
        self.image = cv.hconcat([self.image1, self.image2])

    def show_image(self, image=0):
        if image == 1:
            plt.imshow(cv.cvtColor(self.image1, cv.COLOR_BGR2RGB))
        elif image == 2:
            plt.imshow(cv.cvtColor(self.image2, cv.COLOR_BGR2RGB))
        elif image == 0:
            plt.imshow(cv.cvtColor(self.image, cv.COLOR_BGR2RGB))

    def find_corners(self, sigma):
        self.current_sigma = sigma
        corners1 = Harris.find_harris_corners(self, self.image1, sigma)
        corners2 = Harris.find_harris_corners(self, self.image2, sigma)

        if len(corners1) <= len(corners2):
            self.cornersA = corners1
            self.cornersB = corners2
            self.imageA = self.image1
            self.imageB = self.image2
        else:
            self.cornersA = corners2
            self.cornersB = corners1
            self.imageA = self.image2
            self.imageB = self.image1
        
    def extract_window(self, image, center, width):
        return image[center.y - width : center.y + width+1, center.x - width : center.x + width+1]

    def compute_ssd(self):
        imageA_gray = cv.cvtColor(self.imageA, cv.COLOR_BGR2GRAY)
        imageB_gray = cv.cvtColor(self.imageB, cv.COLOR_BGR2GRAY)
        pairs = []

        # 21 x 21 pixel window. 
        width = 10 # 21 / 2
        for corner in self.cornersA:
            ssd_min = 10000000
            best_candidate = None
            windowA = self.extract_window(imageA_gray, corner, width)
            for candidate in self.cornersB:
                windowB = self.extract_window(imageB_gray, candidate, width)
                diff = windowA - windowB
                ssd = np.sum(diff * diff)
                if ssd_min > ssd:
                    ssd_min = ssd
                    best_candidate = candidate
            pairs.append((corner, best_candidate))

        image = self.mark_pairs(pairs)
        # plt.imshow(cv.cvtColor(image, cv.COLOR_BGR2RGB))
        savename = f'{self.FILEPATH}{self.savename}_ssd_sigma_{self.current_sigma}{self.FILETYPE}'
        cv.imwrite(savename, image.astype(np.int))

    def compute_ncc(self):
        imageA_gray = cv.cvtColor(self.imageA, cv.COLOR_BGR2GRAY)
        imageB_gray = cv.cvtColor(self.imageB, cv.COLOR_BGR2GRAY)
        pairs = []

        # 21 x 21 pixel window. 
        width = 10 # 21 / 2
        for corner in self.cornersA:
            ncc_max = 0
            best_candidate = None
            windowA = self.extract_window(imageA_gray, corner, width)
            for candidate in self.cornersB:
                windowB = self.extract_window(imageB_gray, candidate, width)
                meanA = np.mean(windowA)
                meanB = np.mean(windowB)
                windowA_new = windowA - meanA
                windowB_new = windowB - meanB
                num = np.sum(windowA_new * windowB_new)
                den = np.sqrt(np.sum(windowA_new*windowA_new) * np.sum(windowB_new*windowB_new))
                ncc = num/den
                if ncc > ncc_max:
                    ncc_max = ncc
                    best_candidate = candidate
            if best_candidate is not None:
                pairs.append((corner, best_candidate))

        image = self.mark_pairs(pairs)
        # plt.imshow(cv.cvtColor(image, cv.COLOR_BGR2RGB))
        savename = f'{self.FILEPATH}{self.savename}_ncc_sigma_{self.current_sigma}{self.FILETYPE}'
        cv.imwrite(savename, image.astype(np.int))

    def mark_pairs(self, pairs):
        radius = 2
        thickness = 1
        
        image = cv.hconcat([self.imageA, self.imageB])
        w = self.imageA.shape[1]
        for pair in pairs:
            color = (0, 0, 255)
            cv.circle(image, pair[0].point, radius, color, thickness)
            cv.circle(image, pair[1].adjust_width(w), radius, color, thickness)
            cv.line(image, pair[0].point, pair[1].adjust_width(w), color, thickness)

        
        return image


In [208]:
images = [
    Image('pair1\\1.jpg', 'pair1\\2.jpg', 'hw4_task1_1'),
    Image('pair2\\1.jpg', 'pair2\\2.jpg', 'hw4_task1_2'),
    Image('pair3\\1.jpg', 'pair3\\2.jpg', 'hw4_task1_3'),
    Image('pair4\\1.jpg', 'pair4\\2.jpg', 'hw4_task1_4'),
    Image('pair5\\1.jpg', 'pair5\\2.jpg', 'hw4_task1_5'),
    Image('pair6\\1.jpg', 'pair6\\2.jpg', 'hw4_task1_6')]

In [210]:
sigmas = [2, 3, 4, 6]
for image in images:
    for sigma in sigmas:
        image.load_images()
        image.find_corners(sigma)
        image.compute_ssd()
        image.load_images()
        image.find_corners(sigma)
        image.compute_ncc()